# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм вида: `<дата> - <заголовок> - <ссылка>`

### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.  

Итоговый датафрейм формировать со столбцами: `<дата> - <заголовок> - <ссылка> - <текст_статьи>`


In [83]:
from bs4 import BeautifulSoup
import requests, time
from fake_useragent import UserAgent
import pandas as pd

class Connect:
  
    def __init__(self):
        self.headers = headers = {'User-Agent': UserAgent().chrome} #.random
        self.hooks = {'response': self.response_info}
        self.session = self.get_session()
        
    def response_info(self,r, *args, **kwargs):
        print(r.status_code, r.url)
        #print(r.text)
        
    def get_session(self):
        return requests.Session()
    
    def get_soup(self,URL):
        res = self.session.get(URL, headers=self.headers,hooks=self.hooks)
        time.sleep(0.2)
        return BeautifulSoup(res.text,'html.parser')

#params
KEYWORDS = ['python', 'парсинг']
source = 'https://habr.com/ru/all/'


In [84]:
def get_short_news(soup, ses):
    '''
    Возвращаем датафрейм со всеми новостями со страницы
    <дата> - <заголовок> - <ссылка> - <текст_статьи>
    '''
    all_data = pd.DataFrame()
    
    #addr_list = list(map(lambda x: x.find('article').get('id'),ar_list))
    #page_count = max(list(map(lambda x: x.text.strip(),page_list)))
    #page_count
    
    ar_list = soup.find_all('article',class_='tm-articles-list__item')
    for item in ar_list:
        date =  item.find('time').get('title') 
        title =  item.find('h2').find('span').text
        ref = 'https://habr.com' + item.find('a',class_='tm-article-snippet__title-link').get('href')
        #desc = item.select('div[class*="article-formatted-body article-formatted-body_version-"]')[0].text.strip()
        all_text = get_all_text(soup = ses.get_soup(ref))
        
        row = {'date':date, 'header':title, 'ref':ref, 'text':all_text}
        
        all_data = pd.concat([all_data,pd.DataFrame([row])],sort=False)
        #all_data = all_data.reindex(['date','header','ref','text'], axis=1)
    return all_data

In [85]:
def get_all_text(soup):
    all_text = soup.find('div',xmlns='http://www.w3.org/1999/xhtml').text
    return all_text

In [86]:
ses = Connect()
soup = ses.get_soup(source)
soup

df_news = get_short_news(soup,ses)
news_filt = df_news[df_news['text'].str.lower().str.split(expand = True).isin(KEYWORDS).any(1)]
news_filt

200 https://habr.com/ru/all/
200 https://habr.com/ru/company/selectel/blog/570016/
200 https://habr.com/ru/post/571798/
200 https://habr.com/ru/post/571796/
200 https://habr.com/ru/company/samsung/blog/571264/
200 https://habr.com/ru/post/530488/
200 https://habr.com/ru/post/570780/
200 https://habr.com/ru/post/571758/
200 https://habr.com/ru/post/571784/
200 https://habr.com/ru/company/audiomania/blog/571750/
200 https://habr.com/ru/post/571782/
200 https://habr.com/ru/company/ruvds/blog/570744/
200 https://habr.com/ru/company/skillfactory/blog/570586/
200 https://habr.com/ru/post/571748/
200 https://habr.com/ru/company/productivity_inside/blog/571776/
200 https://habr.com/ru/post/571772/
200 https://habr.com/ru/post/571770/
200 https://habr.com/ru/post/571764/
200 https://habr.com/ru/post/571762/
200 https://habr.com/ru/post/569552/
200 https://habr.com/ru/company/ruvds/blog/569814/


,date,header,ref,text
0,"2021-08-08, 16:57",Технический подход к пониманию интерфейсов моз...,https://habr.com/ru/company/skillfactory/blog/...,Изображение оценки источника с использованием ...


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: `<дата утечки> - <источник утечки> - <описание утечки>`  

**Подсказка**: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [164]:
import json
from bs4 import BeautifulSoup
import requests, time
import pandas as pd

source='https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {
    'Accept': 'application/json, text/plain, */*'
    ,'Accept-Encoding':'gzip, deflate, br'
    ,'Accept-Language': 'ru,en;q=0.9,en-GB;q=0.8,en-US;q=0.7'
    ,'Connection': 'keep-alive'
    ,'Content-Length': '35'
    ,'Content-Type': 'application/json;charset=UTF-8'
    ,'Host': 'identityprotection.avast.com'
    ,'Origin': 'https://www.avast.com'
    ,'Referer': 'https://www.avast.com/'
    ,'sec-ch-ua': 'Chromium;v=92, Not A;Brand;v=99, Microsoft Edge;v=92'
    ,'sec-ch-ua-mobile': '?0'
    ,'Sec-Fetch-Dest': 'empty'
    ,'Sec-Fetch-Mode': 'cors'
    ,'Sec-Fetch-Site': 'same-site'
    ,'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36 Edg/92.0.902.62'
    ,'Vaar-Header-App-Build-Version': '1.0.0'
    ,'Vaar-Header-App-Product': 'hackcheck-web-avast'
    ,'Vaar-Header-App-Product-Name': 'hackcheck-web-avast'
    ,'Vaar-Version': '0'
}
payload = {
    'emailAddresses': ['test@mail.ru','xxx@x.ru', 'yyy@y.com']
}

session = requests.Session()
res = session.post(source, headers=headers,json=payload)


In [165]:
df = pd.DataFrame()
site_json=json.loads(res.text)
for item in site_json['breaches']:
    date = site_json['breaches'][item]['publishDate']
    site = site_json['breaches'][item]['site']
    description = site_json['breaches'][item]['description']
    
    row = {'date':date, 'site':site, 'description':description}
    df = pd.concat([df,pd.DataFrame([row])],sort=False)
df

,date,description,site
0,2019-03-20T00:00:00Z,"In July 2018, Bookmate was allegedly breached....",bookmate.com
0,2019-02-28T00:00:00Z,"In July 2018, 500px's database was allegedly b...",500px.com
0,2019-03-07T00:00:00Z,"In July 2018, ShareThis's database was alleged...",sharethis.com
0,2019-03-07T00:00:00Z,"In December 2018, Dubsmash's database was alle...",dubsmash.com
0,2019-02-28T00:00:00Z,"In July 2018, 8fit's database was allegedly br...",8fit.com
0,2019-03-14T00:00:00Z,"In May 2018, Artsy's database was allegedly br...",artsy.net
0,2019-03-20T00:00:00Z,"In January 2019, Roll20 allegedly breached. Th...",roll20.net
0,2019-03-14T00:00:00Z,"In June 2018, Streeteasy's database was allege...",streeteasy.com
0,2019-05-23T00:00:00Z,"In 2017, social network LiveJournal's database...",livejournal.com
0,2019-06-13T00:00:00Z,"In May 2019, graphic-design site Canva's datab...",canva.com


### Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)  
```
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ  
```

В итоге должен формироваться датафрейм со столбцами: `<дата поста> - <текст поста>`

In [166]:
import json
from bs4 import BeautifulSoup
import requests, time
from datetime import date

source='https://api.vk.com/method/wall.get'
payload = {
    'access_token': '',
    'domain' : 'netology',
    'v':'5.52',
    'count':50
}
def response_info(r, *args, **kwargs):
    print(r.status_code, r.url)

hooks = {'response': response_info} 

session = requests.Session()
res = session.post(source,data=payload,hooks=hooks)
site_json=json.loads(res.text)

200 https://api.vk.com/method/wall.get


In [173]:
df = pd.DataFrame()
for item in site_json['response']['items']:
    post_date = date.fromtimestamp(item['date'])
    post_text = item['text']
    row = {'date':post_date,'text':post_text}
    df = pd.concat([df,pd.DataFrame([row])],sort=False)
df

,date,text
0,2021-08-06,📅 Составляем список дел до конца лета \n \nДо ...
0,2021-08-08,*партнёрский пост*\n\nКаждый четвёртый выпускн...
0,2021-08-08,Мы очень мало знаем о своём мозге 🤯 \nНескольк...
0,2021-08-07,Мы часто делимся историями роста наших студент...
0,2021-08-07,Мечтаете попасть в реки? 🤪 \n \nСобрали статьи...
0,2021-08-06,"Аналитика не такая сложная, если в ней начать ..."
0,2021-08-05,"Доброго времени суток! 😊 \n \nРассказываем, ка..."
0,2021-08-05,Перемены бывают в жизни каждого из нас: сменит...
0,2021-08-05,Берём любимые снеки и идём смотреть новый роли...
0,2021-08-04,У нас сегодня Нето-викторина 🤩\n\nМы подготови...
